kaggle.json authentication file needs to be present in $HOME/.kaggle/kaggle.json to download dataset using CLI

In [1]:
!mkdir -p data
!kaggle datasets download -d finalepoch/medical-ner -p data/ --unzip

  0%|                                               | 0.00/26.2k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 26.2k/26.2k [00:00<00:00, 1.12MB/s]


In [3]:
json_file = './data/Corona2.json'

In [4]:
import json

In [37]:
with open(json_file, "r") as f:
    data = json.load(f)
data.keys()

dict_keys(['examples'])

In [38]:
data = data['examples']

In [40]:
sample_record = data[0]

In [42]:
sample_record.keys()

dict_keys(['id', 'content', 'metadata', 'annotations', 'classifications'])

In [52]:
print(f"id :{sample_record['id']}")

id :18c2f619-f102-452f-ab81-d26f7e283ffe


In [53]:
print(f"content :{sample_record['content']}")

content :While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]

Diosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.

Racecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]


In [54]:
print(f"metadata :{sample_record['metadata']}")

metadata :{}


In [56]:
print(f"classifications :{sample_record['classifications']}")

classifications :[]


In [60]:
for annotation in sample_record['annotations']:
    for key in annotation.keys():
        print(f"{key}: {annotation[key]}")
    break

id: 0825a1bf-6a6e-4fa2-be77-8d104701eaed
tag_id: c06bd022-6ded-44a5-8d90-f17685bb85a1
end: 371
start: 360
example_id: 18c2f619-f102-452f-ab81-d26f7e283ffe
tag_name: Medicine
value: Diosmectite
correct: None
human_annotations: [{'timestamp': '2020-03-21T00:24:32.098000Z', 'annotator_id': 1, 'tagged_token_id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed', 'name': 'Ashpat123', 'reason': 'exploration'}]
model_annotations: []


In [62]:
sample_record['content'][360:371]

'Diosmectite'

https://www.drugs.com/international/diosmectite.html

## Checking all the annotations mentioned in the sample record

In [133]:
text = sample_record['content']

for annotation in sample_record['annotations']:
    start = annotation['start']
    end = annotation['end']
    entity = text[start:end].strip()
    if entity:
        print(f"{entity},{annotation['tag_name']}")

Diosmectite,Medicine
aluminomagnesium silicate,Medicine
diarrhea,MedicalCondition
kaopectate,Medicine
bismuth compounds,Medicine
Pepto-Bismol,Medicine
diarrhea,MedicalCondition
chemotherapy,Medicine
constipation,MedicalCondition
loperamide,Medicine
diarrhea,MedicalCondition
flatulence,MedicalCondition
loperamide,Medicine
diarrhea,MedicalCondition
diarrhea,MedicalCondition
Racecadotril,Medicine
diarrhea,MedicalCondition


### Finding all the unique tags

In [138]:
tags = []
for record in data:
    for annotation in record['annotations']:
        curr_tag = annotation['tag_name']
        if curr_tag not in tags:
            tags.append(curr_tag)
tags

['Medicine', 'MedicalCondition', 'Pathogen']

### Tranforming Data